# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Правильно делаешь, что добавляешь краткое описание задачи. Это поможет быстро вспомнить, о чем проект, если ты к нему вернешься спустя какое-то время.
</div>

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
try:
    data=pd.read_csv('/datasets/data.csv')
except:
    data=pd.read_csv('data.csv')
data.info()
pd.DataFrame(round(data.isna().mean()*100)).style.background_gradient('coolwarm')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,0
children,0
days_employed,10
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


**Вывод**


Выгрузили данные, и прочитали их, для наглядности определили процент пропусков.

Имеются пропущенные значения в столбцах days_employed и total_income, отрицательные значения, а также запись разного регистра в столбце education и family_status

<div class="alert alert-info"><font color='red'>
    <h2> Комментарий </h2>

    Огромные значения стажа мошгли появиться при ошибочном выборе формата даты(год,месяц,день вместо день,месяц,год) , опечатка, перевод знчений в unixtime.
    
    
    
         




## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data['education']=data['education'].str.lower()
data['family_status']=data['family_status'].str.lower()



def non_nan(df, value, category):
    
    for row in data[category].unique():
        df.loc[(data[value].isna())&(df[category]==row),value]= \
        df.loc[data[category]==row, value].median()
    return df


In [3]:
total_non = non_nan(data,'total_income','income_type')

emploe_non = non_nan(data,'days_employed','dob_years')


In [4]:

data.pivot_table(index='income_type', columns='education', values='total_income', aggfunc='median')

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
income_type,,,,,
безработный,202722.511368,NaN,NaN,59956.991984,NaN
в декрете,NaN,NaN,NaN,53829.130729,NaN
госслужащий,157982.545567,150447.935283,156266.846399,144351.310834,111392.231107
компаньон,190976.633414,150100.960964,172357.950966,167766.595389,NaN
пенсионер,135866.925526,107398.699119,118514.486412,118514.486412,177088.845999
предприниматель,499163.144947,NaN,NaN,NaN,NaN
сотрудник,155103.948747,131629.331952,144499.678153,142594.396847,198570.757322
студент,98201.625314,NaN,NaN,NaN,NaN


In [6]:


# То же самое.
data.groupby(['income_type', 'education']).agg({'total_income':'median'})

total_income
income_type     education                         
безработный     высшее               202722.511368
                среднее               59956.991984
в декрете       среднее               53829.130729
госслужащий     высшее               157982.545567
                начальное            150447.935283
                неоконченное высшее  156266.846399
                среднее              144351.310834
                ученая степень       111392.231107
компаньон       высшее               190976.633414
                начальное            150100.960964
                неоконченное высшее  172357.950966
                среднее              167766.595389
пенсионер       высшее               135866.925526
                начальное            107398.699119
                неоконченное высшее  118514.486412
                среднее              118514.486412
                ученая степень       177088.845999
предприниматель высшее               499163.144947
сотрудник       высшее               155103.948747
                начальное            131629.331952
                неоконченное высшее  144499.678153
                среднее              142594.396847
                ученая степень       198570.757322
студент         высшее                98201.625314

In [7]:

data.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

education                      высшее      начальное  неоконченное высшее  \
income_type     gender                                                      
безработный     F       202722.511368            NaN                  NaN   
                M                 NaN            NaN                  NaN   
в декрете       F                 NaN            NaN                  NaN   
госслужащий     F       150447.935283  128107.169292        150447.935283   
                M       204717.884325  190966.659534        166424.838494   
компаньон       F       174369.537430  152336.943284        172357.950966   
                M       217768.009290  150100.960964        179370.487646   
                XNA               NaN            NaN        203905.157261   
пенсионер       F       132086.122711  102598.653164        118514.486412   
                M       150246.754511  114068.787524        124667.471301   
предприниматель F       499163.144947            NaN                  NaN   
                M       499163.144947            NaN                  NaN   
сотрудник       F       144146.551473  124397.467932        142594.396847   
                M       179564.658485  142594.396847        175068.921286   
студент         M        98201.625314            NaN                  NaN   

education                     среднее  ученая степень  
income_type     gender                                 
безработный     F                 NaN             NaN  
                M        59956.991984             NaN  
в декрете       F        53829.130729             NaN  
госслужащий     F       135073.137857   111392.231107  
                M       160213.467715             NaN  
компаньон       F       155725.928731             NaN  
                M       172357.950966             NaN  
                XNA               NaN             NaN  
пенсионер       F       118514.486412   255425.196556  
                M       118514.486412    98752.495442  
предприниматель F                 NaN             NaN  
                M                 NaN             NaN  
сотрудник       F       132635.207938             NaN  
                M       147530.985003   198570.757322  
студент         M                 NaN             NaN

In [8]:


qq = data.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

# Доступ к полям.
qq.loc[('студент', 'M')]['высшее']

98201.62531401133

In [9]:

def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице qq нужную медиану.
    '''
    try:
        return qq.loc[(income_type, gender)][education]
    except:
        return 'Не найдено'
    
print(super_fillna_func('студент', 'M','высшее') ) 

98201.62531401133


In [10]:


print(super_fillna_func('ревьюер', 'F','высшее') )

Не найдено


In [12]:

# Так сработает apply.
data.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

0        144146.551473
1        132635.207938
2        147530.985003
3        147530.985003
4        118514.486412
             ...      
21520    155725.928731
21521    118514.486412
21522    147530.985003
21523    147530.985003
21524    132635.207938
Length: 21525, dtype: float64

In [ ]:

# Вот так применяем к таблице.

# Запишем в новый столбец.
data['new_income'] = data.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

# Пандас сам заменит пропуски значениями из нового столбца в той же строке.
data['total_income'] = data['total_income'].fillna(data['new_income'])

In [ ]:


# А вот так мы столбец создадим, но сохранять не будем. Замену произведем в воздухе:
data['total_income'] = data['total_income'].fillna(data.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1))

In [ ]:


data['total_income'] = data.groupby(['income_type','gender', 'education'])['total_income'].apply(lambda x: x.fillna(x.median()))
   
# Оно же, но без лямбды:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type','gender', 'education'])['total_income'].transform('median'))

In [4]:

pd.DataFrame(round(data.isna().mean()*100)).style.background_gradient('coolwarm')


,0
children,0
days_employed,0
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

**1**.Пропущенные значения в столбцах days_employed и total_income при помощи функции *non_nan* были заменены на медиальные значения с учетом коррелирующих столбцов.

**2**.Столбцы education и family_status были приведены к общему регистру(нижнему) методом  *str.lower()*

**3**.Возможная причина появления пропусков человеческий фактор(отказ от заполнения неизвестной информации), ошибка форматирования или технические проблемы

### Замена типа данных

In [6]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data['dob_years'] = data['dob_years'].astype(int)


**Вывод**

Для изменения типов строк был использован метод *astype()* по причине простоты записи(альтернатива - метод *to_numeric()*)

### Обработка дубликатов

In [7]:
data.duplicated().sum()

71

In [8]:
data=data.drop_duplicates().reset_index(drop=True)
display(data.duplicated().sum())

0

In [9]:
data['children']=data['children'].replace(20,2)
data['children']=data['children'].replace(-1,1)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

**1**.Методом *drop_duplicates* избавились от явных дубликатов и перезапустили индексацию (*reset_index*)

**2**.Заменили выпадающие значения из столбца **children** на наиболее подходящие методом *replace*

Причины ворзникновения могут быть разнообразными, например дублирование строк при объединении таблиц, либо ошибочноен повтороное заведении информации о клиентах сотрудниками, в столбце **children** скорее всего ошибки пользователей.

### Лемматизация

In [11]:
from pymystem3 import Mystem
m = Mystem()
data['lem_purpose'] = data['purpose'].apply(m.lemmatize)


In [12]:
from collections import Counter
count=data['lem_purpose'].apply(Counter)
count.sum()

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

In [13]:
def debt_purpose(var):
    if 'свадьба' in var:
        return 'свадьба'
    if 'недвижимость' in var:
        return 'недвижимость'
    if 'жилье' in var:
        return 'недвижимость'
    if 'автомобиль'in var:
        return 'автомобиль'
    if 'образование' in var:
        return 'образование'
    return 'прочие цели'
data['new_purpose']=data['lem_purpose'].apply(debt_purpose)

**Вывод**

**1**.Импортировали *Mystem* из библиотеки **pymystem3** для проведения лемматизации

**2**.Прошлись по столбцу **purpose** методом *apply* 

**3**.Для подсчета лемм импортировали *Counter* из **collections**

**4**.Методом *apply* подсчитали все значения

**5**.Написали функцию *debt_purpose* для групировки лемм по общим значениям и сохранили в новый столбец **new_purpose**

### Категоризация данных

In [14]:
def fin_fan(id,deb,value1,value2):
    fin_log = data[[id,deb]]
    fin_dict = data[[id,value1,value2]]
    fin_dict = fin_dict.drop_duplicates().reset_index(drop=True)
    fin_log=fin_log.groupby(id).mean().sort_values(deb)
    return fin_log    

fin_fan('family_status_id','debt','children','total_income')


,debt
family_status_id,
2,0.065693
3,0.071130
0,0.075452
1,0.093471
4,0.097509


In [15]:
fam_dict = pd.Series(data['family_status'].unique(),index=data['family_status_id'].unique()).to_dict()
educ_dict = pd.Series(data['education'].unique(),index=data['education_id'].unique()).to_dict()
fam_dict


{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'не женат / не замужем'}

In [16]:


dict(zip(data['family_status_id'],data['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'не женат / не замужем'}

In [17]:
fin_fan('children','debt','family_status','total_income')


,debt
children,
5,0.000000
0,0.075438
3,0.081818
1,0.091658
2,0.094925
4,0.097561


In [18]:
fin_fan('total_income','debt','family_status','children')

,debt
total_income,
145891,0.0
178101,0.0
178104,0.0
178120,0.0
178129,0.0
...,...
214263,1.0
214448,1.0
214462,1.0


**Вывод**

**1**.Написал функцию для создания "словарей"

**2**.Отдельно написал простой словарь для нагладности

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
def end_non(var1,var2):
    
    solution = data.groupby(var1,as_index=False)[[var2]].agg(['sum','count','mean',lambda x: str(round(x.mean()*100,2)) +'%'])
    return solution
end_non('children','debt')

debt                            
           sum  count      mean <lambda_0>
children                                  
0         1063  14091  0.075438      7.54%
1          445   4855  0.091658      9.17%
2          202   2128  0.094925      9.49%
3           27    330  0.081818      8.18%
4            4     41  0.097561      9.76%
5            0      9  0.000000       0.0%

In [29]:

# В кач-ве агрегирования можно любую функцию брать.
data.groupby('children')['debt'].agg(['count', 'sum', lambda x: x.mean()*100])

,count,sum,<lambda_0>
children,,,
0,14091,1063,7.543822
1,4855,445,9.165808
2,2128,202,9.492481
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


In [30]:

# str - чтоб % добавить. Но может и через форматирование.
data.groupby('children')['debt'].agg(['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ])

,count,sum,<lambda_0>
children,,,
0,14091,1063,7.54%
1,4855,445,9.17%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.0%


**Вывод**

Зависимость имеется , заемщики без детей в среднем меньше допускают возникновение задолженности, с увеличением количества детей шанс допустить просрочку увеличивается.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
end_non('family_status','debt')

debt                            
                       sum  count      mean <lambda_0>
family_status                                         
в разводе               85   1195  0.071130      7.11%
вдовец / вдова          63    959  0.065693      6.57%
гражданский брак       388   4151  0.093471      9.35%
женат / замужем        931  12339  0.075452      7.55%
не женат / не замужем  274   2810  0.097509      9.75%

In [32]:


print('Создали словарь:')
display(fam_dict)


print('\n\nСгруппированная таблица. Берем по id, другой столбец удалили:')
a = data.groupby('family_status_id')['debt'].agg(['count', 'sum', 'mean'])
display(a)


# Заменяем
print('\n\nЗаменяем численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': fam_dict})

Создали словарь:


{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'не женат / не замужем'}



Сгруппированная таблица. Берем по id, другой столбец удалили:


,count,sum,mean
family_status_id,,,
0,12339,931,0.075452
1,4151,388,0.093471
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509




Заменяем численные значения по ключу словаря:


,family_status_id,count,sum,mean
0,женат / замужем,12339,931,0.075452
1,гражданский брак,4151,388,0.093471
2,вдовец / вдова,959,63,0.065693
3,в разводе,1195,85,0.071130
4,не женат / не замужем,2810,274,0.097509


**Вывод**

Результаты показывают, что в среднем люди находящиеся в гражданском браке , а также не женатые/не замужние чаще допускают просрочку, зависимость имеется.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
data['quantile_income']=pd.qcut(data['total_income'],q=5)
end_non('quantile_income','debt')


debt                           
                       sum count      mean <lambda_0>
quantile_income                                      
(20666.999, 98537.6]   344  4291  0.080168      8.02%
(98537.6, 132134.4]    361  4291  0.084130      8.41%
(132134.4, 161335.0]   375  4290  0.087413      8.74%
(161335.0, 214618.2]   361  4291  0.084130      8.41%
(214618.2, 2265604.0]  300  4291  0.069914      6.99%

**Вывод**

Зависимость не обнаружена, люди как с низким доходом, так и с высоким допускают просрочку по долговым обязательствам

<div class="alert alert-info"><font color='red'>
    <h2> Комментарий </h2>

    Зависимость имеется, люди со средним показателем дохода чаще допускают просрочки нежели низко доходный и высокодоходный сегмент

- Как разные цели кредита влияют на его возврат в срок?

In [34]:
end_non('new_purpose','debt')


debt                            
              sum  count      mean <lambda_0>
new_purpose                                  
автомобиль    403   4306  0.093590      9.36%
недвижимость  782  10811  0.072334      7.23%
образование   370   4013  0.092200      9.22%
свадьба       186   2324  0.080034       8.0%

**Вывод**

Зависимость имеется, люди берущие займ на прокупку автомобиля и образование чаще всего допускают просрочку.

## Шаг 4. Общий вывод

Для банка целесообразнее всего выдавать займы людям, без детей, находящимсчя в браке или разводе, на покупку недвижимости.

<div class="alert alert-info"><font color='red'>
    <h2> Комментарий </h2>

1.В категории семейного сатауса:
    
    1)Самые надежные заемщики вдовы/вдовцы - 6.57% вероятность допустить просрочку
    2)Самые ненадежные заемщики не женатые/не замужние - 9.75% вероятность просрочки
    
2.В категории общего дохода:
    
    1)Самые надежные заемщики высокодоходный сегмент - 6.99% вероятность допустить просрочку
    2)Самые ненадежные заемщики сегмент среднего дохода - 8.74% вероятность просрочки
    
2.В категории цель кредита:
    
    1)Самые надежные заемщики приобретающие недвижимость - 7.22% вероятность допустить просрочку
    2)Самые ненадежные заемщики приобретающие автомобиль - 9.36% вероятность просрочки



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.